# Rekurrente neuronale Netze

Im vorherigen Modul haben wir reichhaltige semantische Repräsentationen von Text verwendet und einen einfachen linearen Klassifikator auf den Einbettungen aufgebaut. Diese Architektur erfasst die aggregierte Bedeutung der Wörter in einem Satz, berücksichtigt jedoch nicht die **Reihenfolge** der Wörter, da die Aggregationsoperation auf den Einbettungen diese Information aus dem ursprünglichen Text entfernt hat. Da diese Modelle die Wortreihenfolge nicht modellieren können, sind sie nicht in der Lage, komplexere oder mehrdeutige Aufgaben wie Textgenerierung oder Fragebeantwortung zu lösen.

Um die Bedeutung einer Textsequenz zu erfassen, müssen wir eine andere Architektur neuronaler Netze verwenden, die als **rekurrentes neuronales Netz** oder RNN bezeichnet wird. In einem RNN führen wir unseren Satz nacheinander Symbol für Symbol durch das Netzwerk, und das Netzwerk erzeugt einen **Zustand**, den wir dann zusammen mit dem nächsten Symbol erneut in das Netzwerk einspeisen.

Gegeben die Eingabesequenz von Tokens $X_0,\dots,X_n$, erstellt das RNN eine Sequenz von neuronalen Netzwerkblöcken und trainiert diese Sequenz end-to-end mittels Backpropagation. Jeder Netzwerkblock nimmt ein Paar $(X_i,S_i)$ als Eingabe und erzeugt $S_{i+1}$ als Ergebnis. Der finale Zustand $S_n$ oder die Ausgabe $X_n$ wird in einen linearen Klassifikator eingespeist, um das Ergebnis zu erzeugen. Alle Netzwerkblöcke teilen sich die gleichen Gewichte und werden in einem einzigen Backpropagation-Durchlauf end-to-end trainiert.

Da die Zustandsvektoren $S_0,\dots,S_n$ durch das Netzwerk weitergegeben werden, kann es die sequentiellen Abhängigkeiten zwischen Wörtern lernen. Zum Beispiel, wenn das Wort *nicht* irgendwo in der Sequenz erscheint, kann es lernen, bestimmte Elemente innerhalb des Zustandsvektors zu negieren, was zu einer Verneinung führt.

> Da die Gewichte aller RNN-Blöcke im Bild geteilt werden, kann dasselbe Bild als ein Block (rechts) mit einer rekurrenten Rückkopplungsschleife dargestellt werden, die den Ausgabestatus des Netzwerks zurück an den Eingang weitergibt.

Schauen wir uns an, wie rekurrente neuronale Netze uns dabei helfen können, unser Nachrichten-Dataset zu klassifizieren.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## Einfacher RNN-Klassifikator

Im Fall eines einfachen RNN ist jede rekurrente Einheit ein einfaches lineares Netzwerk, das einen zusammengefügten Eingabevektor und Zustandsvektor aufnimmt und einen neuen Zustandsvektor erzeugt. PyTorch repräsentiert diese Einheit mit der Klasse `RNNCell`, und ein Netzwerk solcher Zellen als `RNN`-Schicht.

Um einen RNN-Klassifikator zu definieren, wenden wir zunächst eine Embedding-Schicht an, um die Dimensionalität des Eingabevokabulars zu reduzieren, und fügen dann eine RNN-Schicht darüber hinzu:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Hinweis:** Wir verwenden hier eine untrainierte Einbettungsschicht zur Vereinfachung, aber für noch bessere Ergebnisse können wir eine vortrainierte Einbettungsschicht mit Word2Vec- oder GloVe-Einbettungen verwenden, wie in der vorherigen Einheit beschrieben. Für ein besseres Verständnis könnten Sie den Code anpassen, um mit vortrainierten Einbettungen zu arbeiten.

In unserem Fall verwenden wir einen gepolsterten Datenlader, sodass jede Charge eine Anzahl gepolsterter Sequenzen gleicher Länge enthält. Die RNN-Schicht nimmt die Sequenz von Einbettungstensoren und erzeugt zwei Ausgaben:
* $x$ ist eine Sequenz von RNN-Zellenausgaben bei jedem Schritt
* $h$ ist der finale versteckte Zustand für das letzte Element der Sequenz

Anschließend wenden wir einen vollständig verbundenen linearen Klassifikator an, um die Anzahl der Klassen zu bestimmen.

> **Hinweis:** RNNs sind recht schwierig zu trainieren, da die Anzahl der Schichten, die bei der Rückwärtsausbreitung beteiligt sind, ziemlich groß wird, sobald die RNN-Zellen entlang der Sequenzlänge entrollt werden. Daher müssen wir eine kleine Lernrate wählen und das Netzwerk auf einem größeren Datensatz trainieren, um gute Ergebnisse zu erzielen. Dies kann ziemlich lange dauern, daher wird die Verwendung einer GPU bevorzugt.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## Langzeit-Kurzzeitgedächtnis (LSTM)

Eines der Hauptprobleme klassischer RNNs ist das sogenannte **Problem der verschwindenden Gradienten**. Da RNNs in einem einzigen Backpropagation-Durchlauf Ende-zu-Ende trainiert werden, fällt es ihnen schwer, den Fehler bis zu den ersten Schichten des Netzwerks weiterzuleiten. Dadurch kann das Netzwerk keine Beziehungen zwischen weit entfernten Tokens lernen. Eine Möglichkeit, dieses Problem zu umgehen, besteht darin, eine **explizite Zustandsverwaltung** durch den Einsatz sogenannter **Gates** einzuführen. Zwei der bekanntesten Architekturen dieser Art sind: **Langzeit-Kurzzeitgedächtnis** (LSTM) und **Gated Relay Unit** (GRU).

![Bild, das eine Beispielzelle eines Langzeit-Kurzzeitgedächtnisses zeigt](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

Ein LSTM-Netzwerk ist ähnlich wie ein RNN organisiert, aber es gibt zwei Zustände, die von Schicht zu Schicht weitergegeben werden: der aktuelle Zustand $c$ und der versteckte Vektor $h$. In jeder Einheit wird der versteckte Vektor $h_i$ mit der Eingabe $x_i$ verknüpft, und sie steuern über **Gates**, was mit dem Zustand $c$ geschieht. Jedes Gate ist ein neuronales Netzwerk mit einer Sigmoid-Aktivierung (Ausgabe im Bereich $[0,1]$), das als bitweises Maskieren betrachtet werden kann, wenn es mit dem Zustandsvektor multipliziert wird. Es gibt folgende Gates (von links nach rechts im obigen Bild):
* **Vergessens-Gate**: Es nimmt den versteckten Vektor und bestimmt, welche Komponenten des Vektors $c$ wir vergessen und welche wir durchlassen müssen.
* **Eingabe-Gate**: Es nimmt Informationen aus der Eingabe und dem versteckten Vektor und fügt sie dem Zustand hinzu.
* **Ausgabe-Gate**: Es transformiert den Zustand über eine lineare Schicht mit $\tanh$-Aktivierung und wählt dann einige seiner Komponenten mithilfe des versteckten Vektors $h_i$ aus, um den neuen Zustand $c_{i+1}$ zu erzeugen.

Die Komponenten des Zustands $c$ können als Flags betrachtet werden, die ein- und ausgeschaltet werden können. Zum Beispiel, wenn wir im Sequenzkontext den Namen *Alice* begegnen, könnten wir annehmen, dass es sich um eine weibliche Figur handelt, und das Flag im Zustand setzen, dass wir ein weibliches Substantiv im Satz haben. Wenn wir später auf die Phrase *und Tom* stoßen, setzen wir das Flag, dass wir ein Plural-Substantiv haben. Durch die Manipulation des Zustands können wir also theoretisch grammatikalische Eigenschaften von Satzteilen verfolgen.

> **Hinweis**: Eine großartige Ressource, um die Interna von LSTMs zu verstehen, ist der Artikel [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) von Christopher Olah.

Obwohl die interne Struktur einer LSTM-Zelle komplex erscheinen mag, verbirgt PyTorch diese Implementierung in der `LSTMCell`-Klasse und stellt das `LSTM`-Objekt bereit, um die gesamte LSTM-Schicht darzustellen. Daher wird die Implementierung eines LSTM-Klassifikators der eines einfachen RNNs, das wir oben gesehen haben, ziemlich ähnlich sein:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## Gepackte Sequenzen

In unserem Beispiel mussten wir alle Sequenzen im Minibatch mit Nullvektoren auffüllen. Dies führt zwar zu einem gewissen Speicherverbrauch, aber bei RNNs ist es noch kritischer, dass zusätzliche RNN-Zellen für die aufgefüllten Eingabeelemente erstellt werden. Diese nehmen am Training teil, tragen jedoch keine wichtigen Eingabeinformationen. Es wäre viel besser, das RNN nur bis zur tatsächlichen Sequenzlänge zu trainieren.

Um dies zu erreichen, wurde in PyTorch ein spezielles Format zur Speicherung gepolsterter Sequenzen eingeführt. Angenommen, wir haben ein gepolstertes Eingabe-Minigruppenbatch, das so aussieht:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
Hierbei repräsentiert 0 die gepolsterten Werte, und der tatsächliche Längenvektor der Eingabesequenzen ist `[5,3,1]`.

Um ein RNN effektiv mit gepolsterten Sequenzen zu trainieren, möchten wir die erste Gruppe von RNN-Zellen mit einem großen Minibatch (`[1,6,9]`) starten, dann jedoch die Verarbeitung der dritten Sequenz beenden und mit verkleinerten Minibatches (`[2,7]`, `[3,8]`) weitermachen, und so weiter. Eine gepackte Sequenz wird daher als ein einziger Vektor dargestellt – in unserem Fall `[1,6,9,2,7,3,8,4,5]` – und einem Längenvektor (`[5,3,1]`), aus dem wir das ursprüngliche gepolsterte Minibatch leicht rekonstruieren können.

Um eine gepackte Sequenz zu erzeugen, können wir die Funktion `torch.nn.utils.rnn.pack_padded_sequence` verwenden. Alle rekurrenten Schichten, einschließlich RNN, LSTM und GRU, unterstützen gepackte Sequenzen als Eingabe und erzeugen gepackte Ausgaben, die mit `torch.nn.utils.rnn.pad_packed_sequence` dekodiert werden können.

Um eine gepackte Sequenz erzeugen zu können, müssen wir den Längenvektor an das Netzwerk übergeben. Daher benötigen wir eine andere Funktion, um Minibatches vorzubereiten:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

Das tatsächliche Netzwerk wäre dem oben genannten `LSTMClassifier` sehr ähnlich, aber der `forward`-Durchlauf erhält sowohl das gepolsterte Minibatch als auch den Vektor der Sequenzlängen. Nach der Berechnung des Embeddings erstellen wir eine gepackte Sequenz, geben sie an die LSTM-Schicht weiter und entpacken anschließend das Ergebnis.

> **Hinweis**: Tatsächlich verwenden wir das entpackte Ergebnis `x` nicht, da wir die Ausgabe aus den versteckten Schichten für die folgenden Berechnungen nutzen. Daher könnten wir das Entpacken in diesem Code vollständig entfernen. Der Grund, warum wir es hier belassen, ist, dass Sie diesen Code bei Bedarf leicht anpassen können, falls Sie die Netzwerkausgabe in weiteren Berechnungen verwenden möchten.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **Hinweis:** Sie haben möglicherweise den Parameter `use_pack_sequence` bemerkt, den wir an die Trainingsfunktion übergeben. Derzeit erfordert die Funktion `pack_padded_sequence`, dass der Längensequenz-Tensor auf dem CPU-Gerät ist, und daher muss die Trainingsfunktion vermeiden, die Längensequenz-Daten während des Trainings auf die GPU zu verschieben. Sie können die Implementierung der Funktion `train_emb` in der Datei [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py) einsehen.


## Bidirektionale und mehrschichtige RNNs

In unseren Beispielen haben alle rekurrenten Netzwerke in eine Richtung gearbeitet, von Anfang bis Ende einer Sequenz. Das erscheint natürlich, da es der Art und Weise ähnelt, wie wir lesen und Sprache hören. Allerdings haben wir in vielen praktischen Fällen zufälligen Zugriff auf die Eingabesequenz, weshalb es sinnvoll sein könnte, die rekurrente Berechnung in beide Richtungen auszuführen. Solche Netzwerke werden als **bidirektionale** RNNs bezeichnet, und sie können erstellt werden, indem man den Parameter `bidirectional=True` an den Konstruktor von RNN/LSTM/GRU übergibt.

Bei der Arbeit mit einem bidirektionalen Netzwerk benötigen wir zwei Zustandsvektoren, einen für jede Richtung. PyTorch kodiert diese Vektoren als einen Vektor mit doppelter Größe, was sehr praktisch ist, da man den resultierenden Zustandsvektor normalerweise an eine vollständig verbundene lineare Schicht übergibt. Man muss lediglich diese Größenänderung berücksichtigen, wenn man die Schicht erstellt.

Ein rekurrentes Netzwerk, sei es eindirektional oder bidirektional, erfasst bestimmte Muster innerhalb einer Sequenz und kann diese entweder im Zustandsvektor speichern oder in die Ausgabe weitergeben. Ähnlich wie bei konvolutionalen Netzwerken können wir eine weitere rekurrente Schicht auf die erste aufbauen, um Muster höherer Ordnung zu erfassen, die aus den von der ersten Schicht extrahierten Mustern niedriger Ordnung bestehen. Dies führt uns zum Konzept des **mehrschichtigen RNN**, das aus zwei oder mehr rekurrenten Netzwerken besteht, wobei die Ausgabe der vorherigen Schicht als Eingabe an die nächste Schicht weitergegeben wird.

![Bild, das ein mehrschichtiges Long-Short-Term-Memory-RNN zeigt](../../../../../lessons/5-NLP/16-RNN/images/multi-layer-lstm.jpg)

*Bild aus [diesem großartigen Beitrag](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) von Fernando López*

PyTorch macht die Konstruktion solcher Netzwerke einfach, da man lediglich den Parameter `num_layers` an den RNN/LSTM/GRU-Konstruktor übergeben muss, um mehrere rekurrente Schichten automatisch zu erstellen. Das bedeutet auch, dass die Größe des Zustandsvektors proportional zunimmt, und man muss dies berücksichtigen, wenn man die Ausgabe der rekurrenten Schichten verarbeitet.


## RNNs für andere Aufgaben

In dieser Einheit haben wir gesehen, dass RNNs für die Sequenzklassifikation verwendet werden können. Tatsächlich können sie jedoch viele weitere Aufgaben bewältigen, wie Textgenerierung, maschinelle Übersetzung und mehr. Diese Aufgaben werden wir in der nächsten Einheit betrachten.



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
